In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [31]:
df_train_target = pd.read_csv('./Data/training_set_labels.csv')
df_train_data = pd.read_csv('./Data/training_set_values.csv')

In [32]:
# Change the name of the status_group in df_train_target 'functional needs repair' to 'non-functional'
# Also change the name of 'non functional' to non-functional
df_train_target['status_group'] = df_train_target['status_group'].replace('functional needs repair', 'non functional')
df_train_target['status_group'] = df_train_target['status_group'].replace('non functional', 'non-functional')

# Check
df_train_target['status_group'].value_counts()

functional        32259
non-functional    27141
Name: status_group, dtype: int64

In [40]:
# encode status_group as 0, 1
df_train_target['status_group'] = df_train_target['status_group'].astype('category')
df_train_target['status_group'] = df_train_target['status_group'].cat.codes
df_train_target['status_group'].value_counts(normalize=True)

0    0.543081
1    0.456919
Name: status_group, dtype: float64

In [34]:
# split df_train_data into numeric and categorical
df_train_data_numeric = df_train_data.select_dtypes(include=['int64', 'float64'])
df_train_data_categorical = df_train_data.select_dtypes(include=['object'])

In [35]:
# How many columns did we have originally?
print('Number of columns in original df_train_data: ', len(df_train_data.columns))

Number of columns in original df_train_data:  40


Gonna chuck everything in and see what I get.

Yeah this is going to take a while.

In [36]:
# Encode the categorical data
df_train_data_categorical = pd.get_dummies(df_train_data_categorical)

In [37]:
# standardize the numerical data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train_data_numeric_scaled = scaler.fit_transform(df_train_data_numeric)

# convert the scaled numerical data into a dataframe
df_train_data_numeric_scaled = pd.DataFrame(df_train_data_numeric_scaled, columns=df_train_data_numeric.columns)

# combine the scaled numerical data with the categorical data
df_train_data = pd.concat([df_train_data_numeric_scaled, df_train_data_categorical], axis=1)

# Check
df_train_data.head().T

,0,1,2,3,4
id,1.512933,-1.320990,-0.130757,1.427676,-0.810478
amount_tsh,1.895665,-0.105970,-0.097630,-0.105970,-0.105970
gps_height,1.041252,1.054237,0.025541,-0.584751,-0.964200
longitude,0.131052,0.094610,0.515158,0.671308,-0.448669
latitude,-1.408791,1.207934,0.639751,-1.849720,1.317271
...,...,...,...,...,...
waterpoint_type_group_communal standpipe,1.000000,1.000000,1.000000,1.000000,1.000000
waterpoint_type_group_dam,0.000000,0.000000,0.000000,0.000000,0.000000
waterpoint_type_group_hand pump,0.000000,0.000000,0.000000,0.000000,0.000000
waterpoint_type_group_improved spring,0.000000,0.000000,0.000000,0.000000,0.000000


And now we have 66181 columns. 

Insayian. 

In [38]:
# Take what I have and split it into train and test and validation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train_data, df_train_target['status_group'], test_size=0.2, random_state=42)

# Check
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

# The sum of the rows is
print('Sum of rows: ', X_train.shape[0] + X_test.shape[0])

# The sum of the rows is the same as the number of rows in the original data

X_train shape:  (47520, 66181)
X_test shape:  (11880, 66181)
y_train shape:  (47520,)
y_test shape:  (11880,)


In [39]:
# Feed into logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(accuracy_score(y_test, y_pred)))

c:\Anaconda3\envs\learn-env\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of logistic regression classifier on test set: 0.80


Probably a lot easier to just do a baseline with the most common.